<a href="https://colab.research.google.com/github/antoniovfonseca/spatialAutocorrelation/blob/main/SpatialAutocorrelation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.Environmental Setup

This section installs and loads the necessary libraries to perform spatial data analysis in Python.
It is designed to be reusable for different spatial projects involving vector data, spatial weights, and spatial statistics.


In [1]:
# Install libraries
!pip install esda -q
!pip install libpysal -q
!pip install geopandas -q
!pip install matplotlib -q
!pip install contextily -q
!pip install mapclassify -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.8/142.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 2.3 MB/s eta 0:00:00


In [2]:
# Geospatial data handling
import geopandas as gpd

# Spatial weights and autocorrelation
from libpysal.weights import Queen, Rook, KNN, DistanceBand, lag_spatial
from esda.moran import Moran, Moran_Local

# Data classification
import mapclassify

# Plotting and visualization
import matplotlib.pyplot as plt

# Optional: add basemaps (e.g. OpenStreetMap)
import contextily as ctx

# Other utilities
import pandas as pd
import numpy as np

## 2.Data Preparation
In this section, we define the input vector data for two different years of interest,
commonly used for land use / land cover change and irrigation analysis.
By using variables for years and file paths, this notebook becomes easier to reuse
with different datasets and temporal scenarios.

In [3]:
# Define the reference years to be analyzed
year_1 = 1990
year_2 = 2020

In [4]:
# Define file paths to the vector data for each year
landcover_path_year_1= "/content/drive/MyDrive/GEOG347/project/input/landcover1990.shp"
landcover_path_year_2= "/content/drive/MyDrive/GEOG347/project/input/landcover2020.shp"
irrigation_path_year_1 = "/content/drive/MyDrive/GEOG347/project/input/irrigation1990.shp"
irrigation_path_year_2 = "/content/drive/MyDrive/GEOG347/project/input/irrigation2020.shp"

In [51]:
# Load GeoDataFrames
gdf_landcover_year_1= gpd.read_file(landcover_path_year_1)
gdf_landcover_year_2 = gpd.read_file(landcover_path_year_2)
gdf_irrig_year_1 = gpd.read_file(irrigation_path_year_1)
gdf_irrig_year_2 = gpd.read_file(irrigation_path_year_2)

In [52]:
# Preview Land Cover Data
print(f"🟫 Land Cover - Year {year_1}:")
display(gdf_landcover_year_1.head())

print(f"🟫 Land Cover - Year {year_2}:")
display(gdf_landcover_year_2.head())

# Preview Irrigation Pivot Data
print(f"💧 Irrigation Pivots - Year {year_1}:")
display(gdf_irrig_year_1.head())

print(f"💧 Irrigation Pivots - Year {year_2}:")
display(gdf_irrig_year_2.head())

🟫 Land Cover - Year 1990:


,DN,geometry
0,4,"POLYGON ((-45.60441 -10.1082, -45.60333 -10.10..."
1,12,"POLYGON ((-45.59821 -10.10847, -45.59794 -10.1..."
2,12,"POLYGON ((-45.59929 -10.10847, -45.59902 -10.1..."
3,4,"POLYGON ((-45.60307 -10.1082, -45.60226 -10.10..."
4,4,"POLYGON ((-45.60145 -10.1082, -45.6001 -10.108..."


🟫 Land Cover - Year 2020:


,DN,geometry
0,39,"POLYGON ((-45.59902 -10.1082, -45.59848 -10.10..."
1,39,"POLYGON ((-45.60684 -10.10901, -45.60657 -10.1..."
2,39,"POLYGON ((-45.60253 -10.10901, -45.60226 -10.1..."
3,39,"POLYGON ((-45.60064 -10.10901, -45.60037 -10.1..."
4,39,"POLYGON ((-45.6001 -10.10928, -45.59983 -10.10..."


💧 Irrigation Pivots - Year 1990:


,DN,geometry
0,1,"POLYGON ((-45.39852 -10.87249, -45.39825 -10.8..."
1,1,"POLYGON ((-45.40364 -11.0544, -45.40175 -11.05..."
2,1,"POLYGON ((-45.80977 -11.66939, -45.8095 -11.66..."
3,1,"POLYGON ((-45.76476 -11.73002, -45.76234 -11.7..."
4,1,"POLYGON ((-45.76853 -11.75023, -45.76773 -11.7..."


💧 Irrigation Pivots - Year 2020:


,DN,geometry
0,1,"POLYGON ((-45.50659 -11.03392, -45.50632 -11.0..."
1,1,"POLYGON ((-45.51952 -11.03473, -45.51737 -11.0..."
2,1,"POLYGON ((-45.46967 -11.05009, -45.46751 -11.0..."
3,1,"POLYGON ((-45.40337 -11.0544, -45.40175 -11.05..."
4,1,"POLYGON ((-45.36995 -11.08782, -45.36726 -11.0..."


## 3.Data Handling

This section handles and standardizes the input datasets. It ensures that all spatial layers:

1. Use the same coordinate reference system (CRS)
2. Have valid geometries
3. Contain consistent geometry types
4. Are ready for spatial operations and analysis

In [53]:
# Check and compare CRS for all GeoDataFrames
print(f"CRS - Land Cover {year_1}:", gdf_landcover_year_1.crs)
print(f"CRS - Land Cover {year_2}:", gdf_landcover_year_2.crs)
print(f"CRS - Irrigation {year_1}:", gdf_irrig_year_1.crs)
print(f"CRS - Irrigation {year_2}:", gdf_irrig_year_2.crs)

# Check geometry types
print("\nGeometry Types:")
print(f"Land Cover {year_1}:", gdf_landcover_year_1.geom_type.unique())
print(f"Land Cover {year_2}:", gdf_landcover_year_2.geom_type.unique())
print(f"Irrigation {year_1}:", gdf_irrig_year_1.geom_type.unique())
print(f"Irrigation {year_2}:", gdf_irrig_year_2.geom_type.unique())

# Check for invalid geometries
print("\nInvalid geometries found:")
print(f"Land Cover {year_1}: {~gdf_landcover_year_1.is_valid}.sum()")
print(f"Land Cover {year_2}: {~gdf_landcover_year_2.is_valid}.sum()")
print(f"Irrigation {year_1}: {~gdf_irrig_year_1.is_valid}.sum()")
print(f"Irrigation {year_2}: {~gdf_irrig_year_2.is_valid}.sum()")

CRS - Land Cover 1990: EPSG:4326
CRS - Land Cover 2020: EPSG:4326
CRS - Irrigation 1990: EPSG:4326
CRS - Irrigation 2020: EPSG:4326

Geometry Types:
Land Cover 1990: ['Polygon']
Land Cover 2020: ['Polygon']
Irrigation 1990: ['Polygon']
Irrigation 2020: ['Polygon']

Invalid geometries found:
Land Cover 1990: 0         False
1         False
2         False
3         False
4         False
          ...  
374998    False
374999    False
375000    False
375001    False
375002    False
Length: 375003, dtype: bool.sum()
Land Cover 2020: 0         False
1         False
2         False
3         False
4         False
          ...  
500608    False
500609    False
500610    False
500611    False
500612    False
Length: 500613, dtype: bool.sum()
Irrigation 1990: 0      False
1      False
2      False
3      False
4      False
       ...  
99     False
100    False
101    False
102    False
103    False
Length: 104, dtype: bool.sum()
Irrigation 2020: 0       False
1       False
2       False
3   

In [54]:
# Define a standard CRS
standard_crs = "EPSG:3857"

In [55]:
# Reproject all layers to the standard CRS
gdf_landcover_year_1 = gdf_landcover_year_1.to_crs(standard_crs)
gdf_landcover_year_2 = gdf_landcover_year_2.to_crs(standard_crs)
gdf_irrig_year_1 = gdf_irrig_year_1.to_crs(standard_crs)
gdf_irrig_year_2 = gdf_irrig_year_2.to_crs(standard_crs)

In [56]:
# Fix invalid geometries
gdf_landcover_year_1["geometry"] = gdf_landcover_year_1.buffer(0)
gdf_landcover_year_2["geometry"] = gdf_landcover_year_2.buffer(0)
gdf_irrig_year_1["geometry"] = gdf_irrig_year_1.buffer(0)
gdf_irrig_year_2["geometry"] = gdf_irrig_year_2.buffer(0)

In [57]:
# Confirm uniform CRS and geometry status
print("CRS check (all layers should match):")
print(gdf_landcover_year_1.crs == gdf_landcover_year_2.crs == gdf_irrig_year_1.crs == gdf_irrig_year_2.crs)

print("\nAny remaining invalid geometries?")
print(f"Land Cover {year_1}: {~gdf_landcover_year_1.is_valid.sum()}")
print(f"Land Cover {year_2}: {~gdf_landcover_year_2.is_valid.sum()}")
print(f"Irrigation {year_1}: {~gdf_irrig_year_1.is_valid.sum()}")
print(f"Irrigation {year_2}: {~gdf_irrig_year_2.is_valid.sum()}")

CRS check (all layers should match):
True

Any remaining invalid geometries?
Land Cover 1990: -375004
Land Cover 2020: -500614
Irrigation 1990: -105
Irrigation 2020: -1425


## 4.Data Processing

This section enriches the land cover and irrigation datasets with attributes needed for spatial analysis.
We first process the land use/land cover data for each year, then spatially relate it to irrigation features
(e.g., pivot presence). These derived variables will be used for spatial autocorrelation analysis in the next sections.


In [58]:
# Optional: compute area of each polygon (in hectares)
gdf_landcover_year_1["area_ha"] = gdf_landcover_year_1.geometry.area / 10_000
gdf_landcover_year_2["area_ha"] = gdf_landcover_year_2.geometry.area / 10_000

In [59]:
# Create binary variable for soybean (class 39)
gdf_landcover_year_1["is_soybean"] = (gdf_landcover_year_1["DN"] == 39).astype(int)
gdf_landcover_year_2["is_soybean"] = (gdf_landcover_year_2["DN"] == 39).astype(int)

In [46]:
# Remove polygons smaller than 0.1 hectare (1,000 m²)
gdf_landcover_year_1 = gdf_landcover_year_1[gdf_landcover_year_1.geometry.area > 1000]
gdf_landcover_year_2 = gdf_landcover_year_2[gdf_landcover_year_2.geometry.area > 1000]

In [47]:
# Simplify geometries with tolerance of 5 meters (adjust as needed)
gdf_landcover_year_1["geometry"] = gdf_landcover_year_1.geometry.simplify(tolerance=5)
gdf_landcover_year_2["geometry"] = gdf_landcover_year_2.geometry.simplify(tolerance=5)

In [40]:
# Sample 1000 polygons for faster testing
gdf_landcover_year_1 = gdf_landcover_year_1.sample(n=1000, random_state=42)
gdf_landcover_year_2 = gdf_landcover_year_2.sample(n=1000, random_state=42)

## 5.Spatial Autocorrelation Analysis
This section performs spatial autocorrelation analysis to identify spatial patterns in the land use and irrigation data.
We will use:
- **Global Moran's I** to assess overall spatial clustering,
- **Local Indicators of Spatial Association (LISA)** to identify clusters and spatial outliers.

### Step 1 – Build Spatial Weights Matrix

In [23]:
from libpysal.weights import Queen, lag_spatial

In [ ]:
# STEP 1 - Remove islands and build Queen contiguity weights
from libpysal.weights import Queen

def remove_islands(gdf, column="is_soybean", max_iter=10):
    for i in range(max_iter):
        print(f"🔁 Iteration {i} — {len(gdf)} features")
        w = Queen.from_dataframe(gdf, silence_warnings=True)
        islands = list(w.islands)
        if not islands:
            print("✅ No more islands found.")
            return gdf, w
        print(f"⚠️  Found {len(islands)} islands. Removing them...")
        gdf = gdf.drop(index=gdf.index[islands])
    print("❌ Max iterations reached.")
    return gdf, Queen.from_dataframe(gdf)

# Apply for both years
gdf_lulc_1990, w_queen_1990 = remove_islands(gdf_landcover_year_1)
gdf_lulc_2020, w_queen_2020 = remove_islands(gdf_landcover_year_2)

# Row-standardize the weights
w_queen_1990.transform = 'r'
w_queen_2020.transform = 'r'



🔁 Iteration 0 — 375003 features


<ipython-input-60-60d0c07ffa07>:7: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf, silence_warnings=True)


### Step 2 – Global Moran’s I

In [49]:
from esda.moran import Moran
import numpy as np

# Run Moran's I
moran_1990 = Moran(gdf_lulc_1990["is_soybean"], w_queen_1990, permutations=999)
moran_2020 = Moran(gdf_lulc_2020["is_soybean"], w_queen_2020, permutations=999)

/usr/local/lib/python3.11/dist-packages/esda/moran.py:258: RuntimeWarning: invalid value encountered in scalar divide
  k = k_num / k_den
/usr/local/lib/python3.11/dist-packages/esda/moran.py:270: RuntimeWarning: invalid value encountered in scalar divide
  return self.n / s0 * inum / self.z2ss
/usr/local/lib/python3.11/dist-packages/esda/moran.py:227: RuntimeWarning: invalid value encountered in divide
  self.z /= sy


In [50]:
# Show results
print("🌐 Global Moran’s I — 1990")
print(f"I statistic: {moran_1990.I:.4f}")
print(f"Expected I: {moran_1990.EI:.4f}")
print(f"p-value (simulation): {moran_1990.p_sim:.4f}")
print(f"z-score: {moran_1990.z[0]:.2f}")  # Accessing z[0] avoids formatting errors

print("\n🌐 Global Moran’s I — 2020")
print(f"I statistic: {moran_2020.I:.4f}")
print(f"Expected I: {moran_2020.EI:.4f}")
print(f"p-value (simulation): {moran_2020.p_sim:.4f}")
print(f"z-score: {moran_2020.z[0]:.2f}")

🌐 Global Moran’s I — 1990
I statistic: nan
Expected I: -0.1250
p-value (simulation): 0.0010
z-score: nan

🌐 Global Moran’s I — 2020
I statistic: -0.5455
Expected I: -0.0909
p-value (simulation): 0.0010
z-score: -0.30


In [33]:
# Just to test Moran’s I computation
gdf_lulc_final["has_irrigation"] = [0, 1] * (len(gdf_lulc_final) // 2) + [0] * (len(gdf_lulc_final) % 2)

In [34]:
from esda.moran import Moran

# Compute Moran's I using the 'has_irrigation' variable
moran_global = Moran(gdf_lulc_final["has_irrigation"], w_queen_final)

# Print results
print("🌐 Global Moran's I")
print(f"I statistic: {moran_global.I:.4f}")
print(f"Expected I: {moran_global.EI:.4f}")
print(f"p-value (simulation): {moran_global.p_sim:.4f}")
print(f"z-score: {moran_global.z:.2f}")


🌐 Global Moran's I
I statistic: 0.3000
Expected I: -0.1250
p-value (simulation): 0.1370


TypeError: unsupported format string passed to numpy.ndarray.__format__